In [1]:
"""
TO DO LIST:
    - evaluate the default algorithm
        - holdout
        - confusion matrix
        - ROC / AUC
    
    - compare the default algorithm with the modified ones for single task (my dataset)
        - get each modified algorithm houldout accuracy and standard deviation (SD) estimates
        - calculate the difference in accuracy and SD between the default model and each of the mdified ones
        - Are the differences statistically significant? -> Run hypothesis tests (paired t-test or wilcoxon)

TO LATER DO WITH GROUP:
    - 
        


"""

'\nTO DO LIST:\n    - evaluate the default algorithm\n        - holdout\n        - confusion matrix\n        - ROC / AUC\n    \n    - compare the default algorithm with the modified ones for single task (my dataset)\n        - get each modified algorithm houldout accuracy and standard deviation (SD) estimates\n        - calculate the difference in accuracy and SD between the default model and each of the mdified ones\n        - Are the differences statistically significant? -> Run hypothesis tests (paired t-test or wilcoxon)\n\nTO LATER DO WITH GROUP:\n    - \n        \n\n\n'

In [2]:
# imports

import pandas as pd
import numpy as np
import openml
from AdaBoostWorkyWorky import AdaBoost
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score ,confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [3]:
# get the data from OpenML
# dataset from Task ID = 3904

suite = openml.study.get_suite(99)
#task_id = 3904
task_id = 10101
task = openml.tasks.get_task(task_id)    
dataset = openml.datasets.get_dataset(task.dataset_id)

X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format="array", target=dataset.default_target_attribute
)

# Visualizar o dataset

df = pd.DataFrame(X, columns=attribute_names)
# converter:
#   0 -> -1
#   1 -> 1
df['target'] = 2*y-1 
df = df.dropna(how='any', axis=0)

X = df.drop(columns=['target'], axis=1)
y = df.target

df
print(y.value_counts())

-1    570
 1    178
Name: target, dtype: int64


In [4]:
# splitting the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1111)

In [5]:
# holdout estimation function

def holdout_estimation(model, x, y, test_size_value=0.3, seed=1111):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size_value, random_state=seed)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(f"Estimated accuracy by holdout {accuracy_score(y_test, y_pred):.5f}, using {model.__class__.__name__}")

In [6]:
# fit and predict using the implementation of the default AdaBoost algorithm
# using cross-validation to get an average of the accuracy of the model

ab = AdaBoost()
holdout_estimation(ab, X, y)

Estimated accuracy by holdout 0.72000, using AdaBoost


In [7]:
for alpha in range(3):
    print("Alpha calc method =", alpha)
    ab = AdaBoost()
    ab.fit(X_train, y_train, alpha_type=alpha)
    prediction = ab.predict(X_test)
    
    print("Values predicted:", np.unique(prediction))
    print(f"Accuracy: ", accuracy_score(y_test, prediction))
    print(f"Precision: ", precision_score(y_test, prediction))
    print(f"Recall: ", recall_score(y_test, prediction))
    print()

Alpha calc method = 0
Values predicted: [-1  1]
Accuracy:  0.7155555555555555
Precision:  0.75
Recall:  0.045454545454545456

Alpha calc method = 1
Values predicted: [-1  1]
Accuracy:  0.7111111111111111
Precision:  1.0
Recall:  0.015151515151515152

Alpha calc method = 2
Values predicted: [-1  1]
Accuracy:  0.7066666666666667
Precision:  0.5
Recall:  0.19696969696969696

